# Обработка ошибок в Python

### Синтаксические ошибки

In [1]:
while True print('Hello world')

SyntaxError: invalid syntax (<ipython-input-1-614901b0e5ee>, line 1)

### Исключения

In [2]:
10 * (1/0)

ZeroDivisionError: division by zero

In [3]:
4 + spam*3

NameError: name 'spam' is not defined

In [4]:
'2' + 2

TypeError: must be str, not int

### Ограничения на работу программы

In [5]:
def summ(*args):
    assert all(list(map(lambda x: isinstance(x, int), args))), \
        "arguments must be int. Passed: {}".format(list(map(type, args)))
    return sum(args)

In [6]:
summ(1, "1")

AssertionError: arguments must be int. Passed: [<class 'int'>, <class 'str'>]

In [7]:
def summ(*args):
    if __debug__:
        if not all(list(map(lambda x: isinstance(x, int), args))):
            raise AssertionError("arguments must be int. Passed: {}".format(list(map(type, args))))
    return a + b

In [8]:
summ(1, "1")

AssertionError: arguments must be int. Passed: [<class 'int'>, <class 'str'>]

### Look before you leap

In [9]:
def summ(*args):
    def process(args):
        processed = []
        for arg in args:
            if isinstance(arg, str):
                processed.append(int(arg))
            elif isinstance(arg, float):
                processed.append(int(arg))
            elif isinstance(arg, (list, dict, tuple)):
                processed.extend(process(arg))
            elif isinstance(arg, bool):
                processed.append(arg)
            elif isinstance(arg, int):
                processed.append(arg)
            # 100 elif
            else:
                print("Exclude undefined type: {}".format(type(arg)))
        return processed
    return sum(process(args))

In [10]:
summ(1, [1, 2, 3], "2")

9

In [11]:
summ(1, [1, 2, 3], "2q")

ValueError: invalid literal for int() with base 10: '2q'

### Easier to ask for forgiveness than permission

In [12]:
def summ(*args):
    def process(arg):
        if isinstance(arg, (list, dict, tuple)):
            return list(map(int, arg))
        return [int(arg)]
    res = []
    for arg in args:
        try:
            res.append(sum(process(arg)))
        except:
            print("Exclude undefined type: {}".format(type(arg)))
    return sum(res)

In [13]:
summ(1, [1, 2, 3], "2")

9

In [14]:
summ(1, [1, 2, 3], "2q")

Exclude undefined type: <class 'str'>


7

### Обработка исключений

In [15]:
try:
    k = 1 / 0
except ZeroDivisionError:
    k = 0
print(k)

0


In [16]:
# запускать не рекомендуется
try:
    for epoch in range(int()):
        print("Epoch {} passed.".format(epoch))
except KeyboardInterrupt:
    print('Training was stoped.')

### try exept else

In [17]:
def zd(a):
    return a / 0

def ve(a):
    # ValueError
    return int(a)

def caller(f, *args):
    try:
        k = f(*args)
    except ZeroDivisionError:
        k = 0
        print("Handled")
    else:
        print(k)


In [18]:
caller(zd, 1)

Handled


In [19]:
caller(ve, "2q")

ValueError: invalid literal for int() with base 10: '2q'

In [20]:
caller(ve, "2")

2


### raise

In [21]:
from functools import wraps
def calc_errors(func):
    @wraps(func)
    def res(*args):
        try:
            return func(*args)
        except :
            res.error_count += 1
            raise
    res.error_count = 0
    return res
@calc_errors
def zd(a):
    return a / 0

In [22]:
zd(1)

ZeroDivisionError: division by zero

In [23]:
zd.error_count

1

### finally

In [24]:
from functools import wraps
from collections import Counter

def calc_call_stats(func):
    @wraps(func)
    def res(*args):
        try:
            result = func(*args)
        except :
            res.counts["errors"] += 1
            raise
        else:
            res.counts["successes"] += 1
        finally :
            res.counts["calls"] += 1
        return result
    res.counts = Counter()
    return res
@calc_call_stats
def zd(a):
    return 1 / a

In [25]:
zd(1)

1.0

In [26]:
zd.counts

Counter({'calls': 1, 'successes': 1})

In [27]:
zd(0)

ZeroDivisionError: division by zero

In [28]:
zd.counts

Counter({'calls': 2, 'errors': 1, 'successes': 1})

### Доступ к объекту исключения

In [29]:
try:
    k = 1 / 0
except Exception as e:
    print("something wrong {}.".format(e))
    raise

something wrong division by zero.


ZeroDivisionError: division by zero

### Несколько except

In [30]:
import sys
 
try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except IOError as err:
    print("I/O  error({0}): {1}".format(err.errno, err.strerror))
except ValueError:
    print("Не могу преобразовать данные в целое.")
except:
    print("Неожиданная ошибка:", sys.exc_info()[0])
    raise

I/O  error(2): No such file or directory


### Порождение исключений

In [31]:
def test(x):
    if not isinstance(x, int):
        raise TypeError("test() arg must be int")
    return x + 5

In [32]:
test("2q")

TypeError: test() arg must be int

### Исключения, определённые пользователем

In [33]:
class MyError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value)

In [34]:
try:
    raise MyError(2*2)
except MyError as e:
    print('Поймано моё исключение со значением:', e.value)

Поймано моё исключение со значением: 4


In [35]:
#Собственные исключения
class ShoeError(Exception):
    """Basic exception for errors raised by shoes"""
    pass
class UntiedShoelace(ShoeError):
    """You could fall"""
    pass

class WrongFoot(ShoeError):
    """When you miss left and right"""
    pass

### raise from

In [36]:
try:
    zd(0)
except ZeroDivisionError as e:
    raise TypeError("actually, type error") from e

TypeError: actually, type error

### Библиотеки для работы с исключениями

In [37]:
import sys

try:
    k = 1 / 0
except Exception as e:
    exc_info = sys.exc_info()
    tr = sys.last_traceback

In [38]:
exc_info

(ZeroDivisionError,
 ZeroDivisionError('division by zero'),
 <traceback at 0x108c69c48>)

In [39]:
tr.tb_frame.f_code

<code object run_code at 0x107b07150, file "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2817>

In [40]:
import traceback

In [41]:
traceback.extract_tb(tr)

[<FrameSummary file /usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py, line 2847 in run_code>,
 <FrameSummary file <ipython-input-36-5bc0b43730c0>, line 4 in <module>>]

# Контектные менеджеры

In [42]:
with open("what_is_love.txt", "r") as infile:
    for line in infile:
        print("> {}".format(line))

FileNotFoundError: [Errno 2] No such file or directory: 'what_is_love.txt'

### Собственные контекстные менеджеры

In [43]:
class File():
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.open_file = open(self.filename, self.mode)
        return self.open_file

    def __exit__(self, *args):
        self.open_file.close()


In [44]:
with File("foo.txt", "w") as infile:
    for _ in range(10000):
        infile.write(’foo’)

SyntaxError: invalid character in identifier (<ipython-input-44-02a18a4d1892>, line 3)

# Модули

In [45]:
import myprinter

In [46]:
myprinter.pprint("Some text")

Run this package on windows: "Some text"


### Импорт модуля

In [47]:
import myprinter

In [48]:
import myprinter as mp

In [49]:
from myprinter import pprint

In [50]:
from myprinter import *

### Пакеты

In [51]:
import os


def print_package_tree(path="printer", tab=""):
    # <your code here>
    print(tab, path.split("\\")[-1])
    if os.path.isdir(path):
        listdir = os.listdir(path)
        if listdir:
            for d in listdir:
                print_package_tree(os.path.join(path, d), tab + "\t")

In [42]:
print_package_tree()

 printer
	 linux2
		 printer.py
	 win32
		 printer.py
		 __pycache__
			 printer.cpython-36.pyc
	 __init__.py
	 __pycache__
		 __init__.cpython-36.pyc


In [43]:
import os
from printer.printer import pprint

In [44]:
pprint("test")

Run this package on windows: "test"


# Регулярные выражения

In [52]:
import re
res = re.match(r'AV', 'AV Analytics Vidhya AV')
print(res)
print(res.group(0))

<_sre.SRE_Match object; span=(0, 2), match='AV'>
AV


In [53]:
print(re.match(r'Analytics', 'AV Analytics Vidhya AV'))

None


In [54]:
re.search(r'Analytics', 'AV Analytics Vidhya AV')

<_sre.SRE_Match object; span=(3, 12), match='Analytics'>

In [55]:
re.findall(r'AV', 'AV Analytics Vidhya AV')

['AV', 'AV']

In [56]:
re.split(r'y', 'Analytics')

['Anal', 'tics']

In [57]:
print(re.split(r'i', 'Analytics Vidhya'))
print(re.split(r'i', 'Analytics Vidhya',maxsplit=1))

['Analyt', 'cs V', 'dhya']
['Analyt', 'cs Vidhya']


In [58]:
re.sub(r'India', 'the World', 'AV is largest Analytics community of India')

'AV is largest Analytics community of the World'

In [59]:
pattern = re.compile('AV')
result = pattern.findall('AV Analytics Vidhya AV')
print(result)
result2 = pattern.findall('AV is largest analytics community of India')
print(result2)

['AV', 'AV']
['AV']
